# Instagram Dataframe Creator

In [1]:
import pandas as pd
import os
from datetime import datetime
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import subprocess
import names
from selenium.webdriver.common.by import By
import time
import math
import random 

In [2]:
email = "maskradiogreece@gmail.com"
password = "code1234567"

In [3]:
def scrap_profile(name,count):
    try:
        command = "instagram-scraper --user " + name + " -u " + email + " -p " + password + " -m " + str(count) + " --cookiejar cookies --media-metadata -t none -d JsonData"  
        subprocess.check_output(command)
    except subprocess.CalledProcessError as e:
        raise Exception("Sorry, papala")

In [4]:
driver = webdriver.Chrome("../Data/chromedriver_win32/chromedriver.exe")
driver.maximize_window()
driver.get("https://www.instagram.com/")
time.sleep(5)
buttons = driver.find_elements_by_tag_name("button")
time.sleep(1)
buttons[2].click()
login_inputs = driver.find_elements_by_tag_name("input")
time.sleep(1)
login_inputs[0].send_keys(email)
time.sleep(1)
login_inputs[1].send_keys(password)
time.sleep(1)
buttons[0].click()
time.sleep(5)
notification_button = driver.find_elements(By.XPATH, '//button[text()="Όχι τώρα"]')
time.sleep(1)
notification_button[0].click()

In [ ]:
while True:
    driver.get("https://www.instagram.com/")
    inputs = driver.find_elements_by_tag_name("input")
    # The 3rd one is the search 
    search = inputs[2]
    random_name = names.get_first_name() 
    search.send_keys(random_name)
    time.sleep(2)
    # Get List of Users
    user_list = driver.find_elements(By.XPATH, '//div[@role="none"]')
    # Create List 
    profile_links = []
    for user in user_list:
        link = user.find_element_by_tag_name("a")
        link = link.get_attribute("href")
        if link.find("explore") == -1:
            profile_links.append(link)
    searched_links = pd.read_csv("./Links.csv")
    searched_links = searched_links["Links"].to_list()
    for link in profile_links:
        name = link.split("/")[3]
        if link in searched_links:
            print(f"SKIPPING ====> USER {name} ALREADY SCRAPED ")
            continue
        driver.get(link);
        try:
            is_private = driver.find_element(By.XPATH, "//*[text()[contains(., 'Αυτός ο λογαριασμός είναι ιδιωτικός')]]" )
            continue 
        except: 
            post = driver.find_element(By.XPATH, "//*[text()[contains(., 'δημοσιεύσεις') or contains(., 'δημοσίευση')]]" )
            num_posts = int(post.find_element_by_tag_name("span").get_attribute("innerHTML").replace(",",""))
            if num_posts < 20:
                print("SKIPPING ====> LESS THAN 10 POSTS")
                continue
            get_num_posts = min(num_posts,200)
            print(f"FETCHING ====> Getting {get_num_posts} from {name}")
            try:
                scrap_profile(name,get_num_posts)
                time.sleep(random.randint(60,180))
            except:
                print(f"There was error on getting {name}")

FETCHING ====> Getting 200 from jamescharles
